### Advanced Lists in Python

This notebook goes beyond the basics of Python lists. We'll cover:
- Creation, heterogeneity, and identity vs equality
- Indexing (positive/negative), slicing, and slice assignment
- Adding/removing items (`append`, `extend`, `insert`, `pop`, `remove`, `del`)
- Sorting (`sorted` vs `list.sort`, keys, reverse, stability)
- Copying, aliasing, shallow vs deep copies (and common pitfalls)
- Comprehensions (with conditions, nested)
- Iteration patterns (`enumerate`, unpacking, `_` placeholder)
- Membership, `index`, `count`
- Concatenation and repetition
- 2D lists (matrix-like data) and the *repetition pitfall*
- Idioms: reversing, rotating, deduplicating while preserving order

Where useful, we include quick checks to illustrate behavior and edge cases.

## 1) Creation, Heterogeneity, Identity vs Equality

In [1]:
# Literal creation, heterogeneous types
l = [10, 3.14, "python", True, None]
l, type(l), len(l)

([10, 3.14, 'python', True, None], list, 5)

In [2]:
# Equality vs identity
a = [1, 2, 3]
b = [1, 2, 3]
c = a  # alias (same object)
a == b, a is b, a is c, id(a), id(b), id(c)

(True, False, True, 2902853603776, 2902853494848, 2902853603776)

## 2) Indexing and Slicing

- Positive and negative indices
- Slices: `seq[start:stop:step]` (any part optional)
- Slices never raise `IndexError`; indexing can.
- Slicing returns a **new list** (copy of the selected portion); indexing returns a single element (no copy for immutable elements; but remember nested mutables!).

In [3]:
seq = [10, 20, 30, 40, 50, 60]
first = seq[0]
last = seq[-1]
middle_slice = seq[1:4]   # 20,30,40
open_ended_left = seq[:3] # 10,20,30
open_ended_right = seq[3:]# 40,50,60
step_slice = seq[::2]     # 10,30,50
reverse_copy = seq[::-1]  # reversed copy
first, last, middle_slice, open_ended_left, open_ended_right, step_slice, reverse_copy, seq

(10,
 60,
 [20, 30, 40],
 [10, 20, 30],
 [40, 50, 60],
 [10, 30, 50],
 [60, 50, 40, 30, 20, 10],
 [10, 20, 30, 40, 50, 60])

**Slice assignment** can replace a range (and can change length!). Useful for in-place edits and efficient splicing without rebuilds of tails/heads by hand.

In [4]:
nums = [1, 2, 3, 4, 5]
nums[1:4] = [20, 30]  # Replace 2,3,4 with two elements (length changes)
nums  # -> [1, 20, 30, 5]

# Inserting via empty slice
nums[2:2] = [25, 26]
nums  # -> [1, 20, 25, 26, 30, 5]

# Deleting via slice assignment to []
nums[1:4] = []
nums  # -> [1, 26, 30, 5]

# Strided slice assignment requires matching lengths
a = [0, 1, 2, 3, 4, 5]
a[::2] = [10, 20, 30]  # Replaces indices 0,2,4
a  # -> [10, 1, 20, 3, 30, 5]

[10, 1, 20, 3, 30, 5]

## 3) Adding and Removing Elements
- `append(x)`: add one item to the end
- `extend(iterable)`: add many items (iterates over the argument)
- `insert(i, x)`: insert at index `i`
- `pop([i])`: remove and return item at index (default last)
- `remove(x)`: remove first occurrence of value `x` (raises `ValueError` if not found)
- `del seq[i:j:k]`: delete by index/slice (in place)

In [5]:
items = ["a", "b"]
items.append("c")                   # ["a","b","c"]
items.extend(["d", "e"])           # ["a","b","c","d","e"]
items.insert(1, "X")               # ["a","X","b","c","d","e"]
last = items.pop()                  # removes "e"
middle = items.pop(2)               # removes "b"
items.remove("X")                   # removes first "X"
del items[1:3]                      # delete a slice in place
items, last, middle                 # Expect (['a', 'd'], 'e', 'b') or similar sequence showing operations worked.

(['a'], 'e', 'b')

## 4) Sorting
- `sorted(iterable, *, key=None, reverse=False)` returns a **new list**.
- `list.sort(*, key=None, reverse=False)` sorts **in place** and returns `None`.
- Sort is **stable**: equal keys preserve original order.
- Use `key` for custom order and complex structures.

In [6]:
words = ["pear", "Apple", "banana", "apricot", "Cherry"]
sorted_case_sensitive = sorted(words)
sorted_case_insensitive = sorted(words, key=str.lower)
words_sorted_inplace = words[:]
words_sorted_inplace.sort(key=len, reverse=True)  # by length, descending
sorted_case_sensitive, sorted_case_insensitive, words_sorted_inplace, words  # words unchanged by sorted(), but changed by .sort() on its copy

(['Apple', 'Cherry', 'apricot', 'banana', 'pear'],
 ['Apple', 'apricot', 'banana', 'Cherry', 'pear'],
 ['apricot', 'banana', 'Cherry', 'Apple', 'pear'],
 ['pear', 'Apple', 'banana', 'apricot', 'Cherry'])

**Sorting complex structures** using `key`:
- Sort list of dicts by a field
- Sort tuples by secondary key
- Leverage stability to do multi-pass sorts (least significant first)

In [7]:
records = [
    {"name": "Alice", "age": 30, "score": 88},
    {"name": "Bob",   "age": 25, "score": 88},
    {"name": "Cara",  "age": 25, "score": 95},
]
by_age = sorted(records, key=lambda r: r["age"])                  # age asc
by_score_desc = sorted(records, key=lambda r: r["score"], reverse=True)

# Multi-pass stable sort: sort by name asc, then by score desc (score primary)
tmp = sorted(records, key=lambda r: r["name"])                   # secondary
stable_multi = sorted(tmp, key=lambda r: r["score"], reverse=True) # primary
by_age, by_score_desc, stable_multi

([{'name': 'Bob', 'age': 25, 'score': 88},
  {'name': 'Cara', 'age': 25, 'score': 95},
  {'name': 'Alice', 'age': 30, 'score': 88}],
 [{'name': 'Cara', 'age': 25, 'score': 95},
  {'name': 'Alice', 'age': 30, 'score': 88},
  {'name': 'Bob', 'age': 25, 'score': 88}],
 [{'name': 'Cara', 'age': 25, 'score': 95},
  {'name': 'Alice', 'age': 30, 'score': 88},
  {'name': 'Bob', 'age': 25, 'score': 88}])

## 5) Copying, Aliasing, Shallow vs Deep Copies
- Assignment creates an **alias** (no copy).
- Shallow copies: `list(x)`, `x[:]`, `x.copy()` — copies the outer list, **not** nested objects.
- Deep copies: `copy.deepcopy(x)` — recursively copy nested structures.
- Pitfall: multiplying nested lists replicates references, not values!

In [8]:
import copy

outer = [[0, 0], [1, 1]]
alias = outer                    # same object
shallow1 = list(outer)
shallow2 = outer[:]              # same as copy()
deep = copy.deepcopy(outer)

outer[0][0] = 99
id(outer), id(alias), id(shallow1), id(shallow2), id(deep), outer, shallow1, shallow2, deep

# Notice shallow copies see the inner mutation (shared inner lists), deep does not.

# Repetition pitfall:
bad = [[0]*3]*4  # 4 references to the SAME inner list
bad[0][1] = 7
good = [[0]*3 for _ in range(4)]
good[0][1] = 7
bad, good  # In 'bad', the 7 appears in every row's column 1; in 'good', only the first row changes.

([[0, 7, 0], [0, 7, 0], [0, 7, 0], [0, 7, 0]],
 [[0, 7, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]])

## 6) Comprehensions (and Conditional/Nested Variants)
- Fast, expressive way to construct lists
- Optional `if` filter
- Nest for simple 2D transforms (but keep readability in mind)

In [9]:
nums = list(range(10))
squares = [n*n for n in nums]
evens = [n for n in nums if n % 2 == 0]
pairs = [(i, j) for i in range(3) for j in range(3) if i != j]
squares, evens, pairs  # quick peek at results to verify intent/shape

([0, 1, 4, 9, 16, 25, 36, 49, 64, 81],
 [0, 2, 4, 6, 8],
 [(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)])

Comprehensions with expressions and guarding against edge cases (e.g., division by zero):

In [10]:
denoms = [3, 2, 1, 0, -1]
safe_inverses = [ (1/d) if d != 0 else None for d in denoms ]
safe_inverses  # None where inverse undefined; avoids ZeroDivisionError with a simple guard.

[0.3333333333333333, 0.5, 1.0, None, -1.0]

## 7) Idiomatic Iteration
- `for x in seq` (simple iteration)
- `enumerate(seq, start=0)` for index + value
- Unpacking tuples per element
- Using `_` as a throwaway variable name for ignored fields
- Avoid mutating a list while iterating **over that same list**; iterate over a copy when necessary (`for x in seq[:]`).

In [11]:
data = [("alice", 3), ("bob", 1), ("cara", 2)]
indexed = [(i, name, count) for i, (name, count) in enumerate(data, start=1)]
indexed

# Ignoring fields example
triples = [(1, 2, 3), (4, 5, 6)]
just_middles = [m for _, m, _ in triples]
just_middles

# Safe remove while iterating: iterate over a copy
vals = [0, 1, 2, 3, 4, 5]
for v in vals[:]:
    if v % 2 == 0:
        vals.remove(v)
vals  # Expect only odd numbers remain

[1, 3, 5]

## 8) Membership, `index`, `count`
- `x in seq` / `x not in seq` — membership test (linear time for lists)
- `seq.index(x)` — index of first occurrence (raises `ValueError` if not found)
- `seq.count(x)` — number of occurrences

For many membership tests, consider a `set` for O(1) average-lookups (but it is unordered and deduplicates).

In [12]:
fruits = ["apple", "banana", "banana", "cherry"]
"banana" in fruits, fruits.index("banana"), fruits.count("banana")

# Safely getting index if present
def safe_index(seq, value, default=-1):
    try:
        return seq.index(value)
    except ValueError:
        return default

safe_index(fruits, "kiwi"), safe_index(fruits, "cherry")  # (-1, position of 'cherry')

(-1, 3)

## 9) Concatenation and Repetition
- `a + b` creates a new list
- `a += b` extends in place
- `a * n` repeats contents (shallow copy of the top level!)
- Beware of repetition with nested lists (see the pitfall section above).

In [13]:
a = [1, 2]
b = [3, 4]
c = a + b            # new list
a += b               # mutate a in place
rep = [0, 1] * 3
a, b, c, rep  # observe changes to 'a' after a += b, and c remains independent.

([1, 2, 3, 4], [3, 4], [1, 2, 3, 4], [0, 1, 0, 1, 0, 1])

## 10) 2D Lists and Common Operations
Treat a list of lists as a simple matrix-like structure:
- Access row/column
- Row/column operations via comprehensions
- Transpose (zip trick)
- Beware repetition pitfall when creating grids.

In [14]:
grid = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
]
first_row = grid[0]
second_col = [row[1] for row in grid]
diag = [grid[i][i] for i in range(len(grid))]
transpose = [list(col) for col in zip(*grid)]
first_row, second_col, diag, transpose  # quick inspection of shapes/values

# Update a whole column by slice assignment on each row
for row in grid:
    row[0:1] = [row[0]*10]  # multiply first column by 10 in-place
grid  # first column scaled

[[10, 2, 3], [40, 5, 6], [70, 8, 9]]

## 11) Handy Idioms
- **Reverse in place**: `l.reverse()`; reversed iterator: `reversed(l)`
- **Rotate** (k to the right): `l[-k:] + l[:-k]` (copy) or slice-assign in place
- **Unique while preserving order**:
  - For hashables: keep a `set` of seen
  - For unhashables: use a tuple conversion or a custom key function if possible
- **Clamp** all numbers to a range using comprehension and `min/max`

In [15]:
# Reverse
r = [1, 2, 3]
r.reverse()  # in place
list(reversed([1,2,3]))  # iterator -> list

# Rotate right by k (copy-based)
def rotate(seq, k):
    if not seq:
        return seq
    k %= len(seq)
    return seq[-k:] + seq[:-k]

rotate([1,2,3,4,5], 2)  # -> [4,5,1,2,3]

# Unique while preserving order (hashable elements)
def unique_preserve_order(seq):
    seen = set()
    out = []
    for x in seq:
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out

unique_preserve_order([1,2,2,3,1,5,3,5])  # -> [1,2,3,5]

# Clamp numbers into [lo, hi]
def clamp_list(nums, lo, hi):
    return [min(hi, max(lo, n)) for n in nums]

clamp_list([-5, 0, 5, 10, 15], 0, 10)  # -> [0,0,5,10,10]

[0, 0, 5, 10, 10]